# Autoformer Model for Wind Power Prediction

This notebook trains the Autoformer model with auto-correlation mechanism for time-series forecasting.

**Key Features:**
- Auto-Correlation Mechanism: Discovers period-based dependencies using FFT
- Series Decomposition: Separates trend and seasonal components
- Efficient for long-term forecasting


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import torch.nn as nn
import importlib

project_root = Path().resolve().parent

# Ensure results directories exist
(project_root / 'results' / 'figures').mkdir(parents=True, exist_ok=True)
(project_root / 'results' / 'metrics').mkdir(parents=True, exist_ok=True)
(project_root / 'data' / 'processed').mkdir(parents=True, exist_ok=True)
sys.path.insert(0, str(project_root / 'src'))

# Import and reload modules
import preprocessing
import models.autoformer
importlib.reload(preprocessing)
importlib.reload(models.autoformer)

from preprocessing import time_aware_split, prepare_sequences, FeatureScaler
from models.autoformer import AutoformerModel
from physics_constraints import PhysicsLoss
from training import set_seed, train_model
from evaluation import compute_metrics, plot_predictions
from torch.utils.data import DataLoader, Dataset

set_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## Load and Prepare Data


In [ ]:
# Load cleaned data
data_path_csv = project_root / 'data' / 'processed' / 'scada_cleaned.csv'
data_path_gz = project_root / 'data' / 'processed' / 'scada_cleaned.csv.gz'
mapping_path = project_root / 'data' / 'processed' / 'feature_mapping.json'

if data_path_gz.exists():
    print(f"Loading compressed data from: {data_path_gz}")
    df = pd.read_csv(data_path_gz, index_col=0, parse_dates=True, compression='gzip')
elif data_path_csv.exists():
    print(f"Loading data from: {data_path_csv}")
    df = pd.read_csv(data_path_csv, index_col=0, parse_dates=True)
else:
    raise FileNotFoundError(
        f"Cleaned data file not found! Please run notebook 01_data_exploration.ipynb first."
    )

if not mapping_path.exists():
    raise FileNotFoundError(f"Feature mapping file not found: {mapping_path}")

with open(mapping_path, 'r') as f:
    feature_mapping = json.load(f)

target_col = feature_mapping['target']
feature_cols = feature_mapping['features']
ws_col = feature_mapping['all_features'].get('wind_speed')

# Split data
train_df, val_df, test_df = time_aware_split(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

# Scale features
scaler = FeatureScaler(method='standard')
X_train_scaled = scaler.fit_transform(train_df[feature_cols])
X_val_scaled = scaler.transform(val_df[feature_cols])
X_test_scaled = scaler.transform(test_df[feature_cols])

y_train = train_df[target_col].values
y_val = val_df[target_col].values
y_test = test_df[target_col].values

print(f"Train: {len(train_df):,}, Val: {len(val_df):,}, Test: {len(test_df):,}")


## Prepare Sequences


In [ ]:
# Prepare sequences for Autoformer
sequence_length = 96  # Autoformer works well with longer sequences
max_train_samples = 500_000
max_val_samples = 100_000
max_test_samples = 100_000

print(f"Preparing sequences with length {sequence_length}...")
print(f"Limiting to {max_train_samples:,} training sequences")

# Convert to float32
X_train_f32 = X_train_scaled.values.astype(np.float32)
y_train_f32 = y_train.astype(np.float32)
X_val_f32 = X_val_scaled.values.astype(np.float32)
y_val_f32 = y_val.astype(np.float32)
X_test_f32 = X_test_scaled.values.astype(np.float32)
y_test_f32 = y_test.astype(np.float32)

X_train_seq, y_train_seq = prepare_sequences(
    X_train_f32, y_train_f32, sequence_length,
    max_samples=max_train_samples, dtype=np.float32
)
X_val_seq, y_val_seq = prepare_sequences(
    X_val_f32, y_val_f32, sequence_length,
    max_samples=max_val_samples, dtype=np.float32
)
X_test_seq, y_test_seq = prepare_sequences(
    X_test_f32, y_test_f32, sequence_length,
    max_samples=max_test_samples, dtype=np.float32
)

print(f"\nTrain sequences: {X_train_seq.shape}")
print(f"Val sequences: {X_val_seq.shape}")
print(f"Test sequences: {X_test_seq.shape}")

# Create simple dataset wrapper for pre-prepared sequences
class PreSequenceDataset(Dataset):
    """Dataset wrapper for pre-prepared sequences."""
    def __init__(self, X_seq, y_seq):
        self.X_seq = X_seq
        self.y_seq = y_seq
    
    def __len__(self):
        return len(self.X_seq)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X_seq[idx]), torch.FloatTensor([self.y_seq[idx]] if self.y_seq.ndim == 1 else self.y_seq[idx])

# Create datasets using pre-prepared sequences (to respect max_samples limits)
train_dataset = PreSequenceDataset(X_train_seq, y_train_seq)
val_dataset = PreSequenceDataset(X_val_seq, y_val_seq)
test_dataset = PreSequenceDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


## Create Autoformer Model


In [ ]:
# Create Autoformer model
model = AutoformerModel(
    input_size=X_train_seq.shape[2],
    d_model=512,
    n_heads=8,
    e_layers=2,
    d_layers=1,
    d_ff=2048,
    dropout=0.05,
    factor=1,  # Auto-correlation factor
    output_size=1,
    seq_len=sequence_length,
    label_len=sequence_length // 2,
    pred_len=1
).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


## Train Autoformer Model


In [ ]:
# Create physics-aware loss (optional)
physics_loss = PhysicsLoss(
    lambda_physics=0.1,
    lambda_negative=1.0,
    lambda_monotonic=0.5
)

# Standard MSE loss
mse_loss = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

# Train model
save_dir = project_root / 'results' / 'checkpoints'
history = train_model(
    model,
    train_loader,
    val_loader,
    mse_loss,
    optimizer,
    epochs=100,
    device=device,
    save_dir=save_dir,
    early_stopping_patience=15
)

# Plot training history
plt.figure(figsize=(12, 5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Autoformer Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
figures_dir = project_root / 'results' / 'figures'
figures_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(figures_dir / 'autoformer_training_history.png', dpi=150)
plt.show()


## Evaluate on Test Set


In [ ]:
# Load best model
checkpoint = torch.load(save_dir / 'best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate on test set
model.eval()
y_pred_test = []
y_test_list = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        predictions = model(batch_X)
        y_pred_test.extend(predictions.cpu().numpy())
        y_test_list.extend(batch_y.numpy())

y_pred_test = np.array(y_pred_test)
y_test_flat = np.array(y_test_list).squeeze()

# Ensure shapes match
if len(y_test_flat) != len(y_pred_test):
    min_len = min(len(y_test_flat), len(y_pred_test))
    y_test_flat = y_test_flat[:min_len]
    y_pred_test = y_pred_test[:min_len]

# Compute metrics
metrics = compute_metrics(y_test_flat, y_pred_test)
print("\nAutoformer Model - Test Set Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")

# Save predictions
predictions_df = pd.DataFrame({
    'true': y_test_flat,
    'predicted': y_pred_test
})
predictions_df.to_csv(project_root / 'results' / 'metrics' / 'autoformer_predictions.csv', index=False)

# Plot predictions
plot_predictions(
    y_test_flat,
    y_pred_test,
    save_path=figures_dir / 'autoformer_predictions.png',
    title='Autoformer: Predictions vs Ground Truth'
)
